
# Source Kernels

* [[No TTA] Cassava Resnext50_32x4d Inference lb0.903](https://www.kaggle.com/piantic/no-tta-cassava-resnext50-32x4d-inference-lb0-903/output)
* [Clean_Inference_Kernel_8xTTA_LB902](https://www.kaggle.com/underwearfitting/clean-inference-kernel-8xtta-lb902/data)
* [Cassava-ensemble-(efnetb3-resnet50)](https://www.kaggle.com/shubham108/cassava-ensemble-efnetb3-resnet50)

# Some Modifications:
batch_size=128

Final prediction = 0.65*predictions + 0.35*np.mean

Result: 101st place on Private (+332 from Public LB)



In [ ]:
#for efficientnet
BATCH_SIZE = 1
image_size = 512
enet_type = ['tf_efficientnet_b4_ns'] * 5
model_path = ['../input/cassavaroots/baseline_cld_fold0_epoch8_tf_efficientnet_b4_ns_512.pth', 
              '../input/cassavaroots/baseline_cld_fold1_epoch9_tf_efficientnet_b4_ns_512.pth', 
              '../input/cassavaroots/baseline_cld_fold2_epoch9_tf_efficientnet_b4_ns_512.pth',
              '../input/cassavaroots/baseline_cld_fold3_epoch5_tf_efficientnet_b4_ns_512.pth',
              '../input/cassavaroots/baseline_cld_fold4_epoch11_tf_efficientnet_b4_ns_512.pth']

In [ ]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
import albumentations
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
from scipy.special import softmax
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm

image_size = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os

OUTPUT_DIR = './'
MODEL_DIR = '../input/cassavaroots/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'
# ====================================================
# CFG for Resnext
# ====================================================
class CFG:
    debug=False
    num_workers=4
    model_name='resnext50_32x4d'
    size=512
    batch_size=128
    seed=1992
    num_classes=5
    target_col='label'
    resnext = 'resnext50_32x4d'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    inference=True
def seed_all(seed: int):
    if not seed:
        seed = 10

    print("[ Using Seed : ", seed, " ]")
    
    os.environ['PYTHONHASHSEED'] = str(seed)  
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed) 
    np.random.seed(seed)
    random.seed(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
seed_all(seed=CFG.seed)

test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test['filepath'] = test.image_id.apply(lambda x: os.path.join('../input/cassava-leaf-disease-classification/test_images', f'{x}'))
#test.head()

# ====================================================
# Dataset for Resnext
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image
    
# ====================================================
# Transforms for Resnext
# ====================================================
import albumentations as A
def get_transforms(*, data):
    if data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
# ====================================================
# ResNext Model
# ====================================================
class CustomResNext(nn.Module):
    def __init__(self, config, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name=config.resnext, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, config.num_classes)
        
    def forward(self, input_neurons):
        output_predictions = self.model(input_neurons)
        return output_predictions
# ====================================================
# Helper functions for Resnext
# ====================================================
def load_state(model_path):
    model = CustomResNext(CFG, pretrained=False)
    try:  # single GPU model_file
        model.load_state_dict(torch.load(model_path)['model'], strict=True)
        state_dict = torch.load(model_path)['model']
    except:  # multi GPU model_file
        state_dict = torch.load(model_path)['model']
        state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}

    return state_dict


def inference(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for state in states:
            model.load_state_dict(state)
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
                print(y_preds)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
        
    probs = np.concatenate(probs)
    return probs

#for Resnext
model = CustomResNext(CFG, pretrained=False)
#model = enet_v2(enet_type[i], out_dim=5)
states = [load_state(MODEL_DIR+f'{CFG.model_name}_fold{fold}.pth') for fold in CFG.trn_fold]
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)


predictions = inference(model, states, test_loader, device)
predictions

In [ ]:
#Transform for efficientnet
transforms_valid = albumentations.Compose([
    albumentations.CenterCrop(image_size, image_size, p=1),
    albumentations.Resize(image_size, image_size),
    albumentations.Normalize()
])

# Directory settings

In [ ]:
# ====================================================
# Directory settings for Resnext
# ====================================================
import os

OUTPUT_DIR = './'
MODEL_DIR = '../input/cassavaroots/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'

# CFG

In [ ]:
# ====================================================
# CFG for Resnext
# ====================================================
class CFG:
    debug=False
    num_workers=4
    model_name='resnext50_32x4d'
    size=512
    batch_size=128    #was 32
    seed=1992
    num_classes=5
    target_col='label'
    resnext = 'resnext50_32x4d'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    inference=True

# Utils

In [ ]:
# ====================================================
# Utils for Resnext
# ====================================================
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


def init_logger(log_file=OUTPUT_DIR+'inference.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

#LOGGER = init_logger()

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [ ]:
def seed_all(seed: int):
    if not seed:
        seed = 10

    print("[ Using Seed : ", seed, " ]")
    
    os.environ['PYTHONHASHSEED'] = str(seed) 
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed) 
    np.random.seed(seed) 
    random.seed(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
seed_all(seed=CFG.seed)

# Data Loading

In [ ]:
test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test['filepath'] = test.image_id.apply(lambda x: os.path.join('../input/cassava-leaf-disease-classification/test_images', f'{x}'))


# Dataset

In [ ]:
# ====================================================
# Dataset for Resnext
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [ ]:
# ====================================================
# Dataset for efficientnet
# ====================================================
class CLDDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        image = cv2.imread(row.filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image']
        
        image = image.astype(np.float32)
        image = image.transpose(2,0,1)
        if self.mode == 'test':
            return torch.tensor(image).float()
        else:
            return torch.tensor(image).float(), torch.tensor(row.label).float()



In [ ]:
#for efficientnet
test_dataset_efficient = CLDDataset(test, 'test', transform=transforms_valid)
test_loader_efficient = torch.utils.data.DataLoader(test_dataset_efficient, batch_size=BATCH_SIZE, shuffle=False,  num_workers=4)

In [ ]:
# ====================================================
# Transforms for Resnext
# ====================================================
def get_transforms(*, data):
    if data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

# MODELS

In [ ]:
# ====================================================
# ResNext Model
# ====================================================
class CustomResNext(nn.Module):
    def __init__(self, config, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name=config.resnext, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, config.num_classes)
        
    def forward(self, input_neurons):
        output_predictions = self.model(input_neurons)
        return output_predictions

In [ ]:
# ====================================================
# EfficientNet Model
# ====================================================
class enet_v2(nn.Module):

    def __init__(self, backbone, out_dim, pretrained=False):
        super(enet_v2, self).__init__()
        self.enet = timm.create_model(backbone, pretrained=pretrained)
        in_ch = self.enet.classifier.in_features
        self.myfc = nn.Linear(in_ch, out_dim)
        self.enet.classifier = nn.Identity()

    def forward(self, x):
        x = self.enet(x)
        x = self.myfc(x)
        return x

In [ ]:
import sys
sys.path.append('../input/hongnangeffnet/gen-efficientnet-pytorch-master-hongnan')
import geffnet

In [ ]:
class configuration:
    effnet = 'tf_efficientnet_b5_ns'
    num_classes=5
config = configuration

In [ ]:
class CustomEfficientNet(nn.Module):
    def __init__(self, config: type, pretrained: bool=True):
        super().__init__()
        self.config = config
        self.model = geffnet.create_model(
        model_name=config.effnet,
        pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, config.num_classes)
        

    def forward(self, input_neurons):
        output_predictions = self.model(input_neurons)
        return output_predictions

# Helper functions

In [ ]:
# ====================================================
# Helper functions for Resnext
# ====================================================
def load_state(model_path):
    model = CustomResNext(CFG, pretrained=False)
    try:  # single GPU model_file
        model.load_state_dict(torch.load(model_path)['model'], strict=True)
        state_dict = torch.load(model_path)['model']
    except:  # multi GPU model_file
        state_dict = torch.load(model_path)['model']
        state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}

    return state_dict


def inference(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for state in states:
            model.load_state_dict(state)
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
                print(y_preds)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
        
    probs = np.concatenate(probs)
    return probs

In [ ]:
# ====================================================
# Helper functions for efficientnet
# ====================================================
def inference_func(test_loader):
    model.eval()
    bar = tqdm(test_loader)

    LOGITS = []
    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            logits = model(x)
            LOGITS.append(logits.cpu())
            PREDS += [torch.softmax(logits, 1).detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()
        LOGITS = torch.cat(LOGITS).cpu().numpy()
    return PREDS

def tta_inference_func(test_loader):
    model.eval()
    bar = tqdm(test_loader)
    PREDS = []
    LOGITS = []

    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
            x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
            x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],0)
            x = x.view(-1, 3, image_size, image_size)
            logits = model(x)
            logits = logits.view(BATCH_SIZE, 8, -1).mean(1)
            PREDS += [torch.softmax(logits, 1).detach().cpu()]
            LOGITS.append(logits.cpu())

        PREDS = torch.cat(PREDS).cpu().numpy()
        
    return PREDS

# inference and Submit

for resnext, the dataloader outputs

    tensor([[[[ 0.1426,  0.3652,  0.7248,  ...,  0.5364,  0.4337, -0.7822],
              [ 0.2796,  0.2282,  0.4508,  ...,  0.5193,  0.5707, -0.3541],
              [ 0.2796,  0.1426,  0.2624,  ...,  0.5878,  0.8447,  0.6563],
              ...,
              [-1.5357, -1.5357, -1.5185,  ..., -1.6384, -1.5870, -1.5528],
              [-1.7069, -1.6555, -1.6042,  ..., -1.8097, -1.7583, -1.7069],
              [-1.8439, -1.7925, -1.7583,  ..., -1.7925, -1.7583, -1.7583]],

             [[ 0.2052,  0.5553,  1.0805,  ...,  1.2731,  1.0630, -0.2325],
              [ 0.3452,  0.4328,  0.8354,  ...,  1.2031,  1.1681,  0.1352],
              [ 0.3978,  0.3627,  0.6254,  ...,  1.1856,  1.3782,  1.0980],
              ...,
              [-1.1253, -1.0903, -1.0903,  ..., -1.7031, -1.6506, -1.6155],
              [-1.2654, -1.2129, -1.1429,  ..., -1.8782, -1.8256, -1.7731],
              [-1.3880, -1.3354, -1.3004,  ..., -1.8606, -1.8256, -1.8256]],

             [[ 0.0431,  0.3568,  0.8274,  ...,  0.2522,  0.3045, -0.7413],
              [ 0.2348,  0.2871,  0.6182,  ...,  0.2348,  0.4265, -0.3404],
              [ 0.3916,  0.3045,  0.5485,  ...,  0.3219,  0.7054,  0.6705],
              ...,
              [-0.9853, -0.9678, -0.9678,  ..., -1.4210, -1.3687, -1.3339],
              [-1.1421, -1.0898, -1.0376,  ..., -1.5779, -1.5256, -1.4733],
              [-1.2816, -1.2293, -1.1944,  ..., -1.5430, -1.5081, -1.5081]]]])

In [ ]:
# ====================================================
# inference
# ====================================================

#for Resnext
model = CustomResNext(CFG, pretrained=False)
states = [load_state(MODEL_DIR+f'{CFG.model_name}_fold{fold}.pth') for fold in CFG.trn_fold]
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)
states[0]

array([[0.04188796, 0.04432843, 0.24820721, 0.03057373, 0.63500273]],
      dtype=float32)

In [ ]:
#for Efficientnet
test_preds = []
for i in range(len(enet_type)):
    model = enet_v2(enet_type[i], out_dim=5)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i]))
    test_preds += [tta_inference_func(test_loader_efficient)]
np.mean(test_preds, axis=0)

array([[0.03165458, 0.02644923, 0.15103994, 0.00727853, 0.7835777 ]],
      dtype=float32)

In [ ]:
pred = 0.65*predictions + 0.35*np.mean(test_preds, axis=0)
test['label'] = softmax(pred).argmax(1)
test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
test.head()